In [1]:
import random
import numpy as np
import time
import matplotlib.pyplot as plt

from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interact
import ipywidgets as widgets
%matplotlib inline


In [2]:
# define phi as f is at time t = 0 - in particular, phi defines border conditions
def phi(x, y, L):
    if x == 0 or x == L:
        return 1
    return 0
vphi = np.vectorize(phi)

In [3]:
def v_n(init, L, beta, n, K):
    somme = 0
    for _ in range(K):
        x = init[0]
        y = init[1]
        coordList = [[x,y]]
        number_non_touches = 0
        for j in range(n):
            coordList.append([x,y])
            if x != 0 and y != 0 and y != L and x != L: # in that case the walk's fate is sealed
                number_non_touches += 1
                rand = random.random()
                # Go with equal probability left, up, right or down
                if 0 <= rand < 0.25:
                    x -= 1
                elif 0.25 <= rand < 0.5:
                    x += 1
                elif 0.5 <= rand < 0.75:
                    y -= 1
                else:
                    y += 1
                
        somme += vphi(np.array([coord[0] for coord in coordList]),np.array([coord[1] for coord in coordList]), L)*np.exp(- beta*np.arange(n+1))
    return somme/K


In [4]:
def approximate_solution(L, beta, K, N):
    arrayImages = np.zeros((N,L,L))
    for i in range(L):
        for j in range(L):
            ls = v_n([i,j], L, beta, N, K)
            for n in range(N):
                arrayImages[n][i][j] = ls[n]
    return arrayImages

In [5]:
def plot_approximate_solution(L, beta, K, N):
    arrayImages = approximate_solution(L, beta, K, N)
    def _show(frame=widgets.IntSlider(min=0,max=len(arrayImages) - 1,step=1,value=0)):
        fig, ax = plt.subplots(figsize=(7,7))
        ax.imshow(arrayImages[frame][:][:].T, origin="lower",
                   extent=[0, 1, 0, 1], cmap="jet")
    return interact(_show)

In [6]:
def test():
    """Main function."""
    L = 30  # discretize with squares of length 1/L
    K = 50  # number of simulations per point to compute the average result
    N = 200  # maximum time
    gamma = 100
    beta = np.log(gamma/(L**2) + 1)
    plot_approximate_solution(L, beta, K, N)
    
    
if __name__ == "__main__":
    test()

interactive(children=(IntSlider(value=0, description='frame', max=199), Output()), _dom_classes=('widget-inter…